In [1]:
import torch
from kobart import get_kobart_tokenizer
from transformers.models.bart import BartForConditionalGeneration
import json
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [2]:
def load_model():
    model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')
    # tokenizer = get_kobart_tokenizer()
    return model

model = load_model()
tokenizer = get_kobart_tokenizer()

using cached model


In [3]:
def summarization(text):
    text = text.replace('\n', '')

    input_ids = tokenizer.encode(text)
    input_ids = torch.tensor(input_ids)
    input_ids = input_ids.unsqueeze(0)
    output = model.generate(input_ids, eos_token_id=1, max_length=30, num_beams=5)
    output = tokenizer.decode(output[0], skip_special_tokens=True)

    return output

In [4]:
TRAIN_SOURCE = 'data/train.json'
TEST_SOURCE = 'data/test.json'
with open(TRAIN_SOURCE) as f:
    TRAIN_DATA = json.loads(f.read())
    
with open(TEST_SOURCE) as f:
    TEST_DATA = json.loads(f.read())

train = pd.DataFrame(columns=['uid', 'title', 'region', 'context', 'summary'])
uid = 1000
for data in TRAIN_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        train.loc[uid, 'uid'] = uid
        train.loc[uid, 'title'] = data['title']
        train.loc[uid, 'region'] = data['region']
        train.loc[uid, 'context'] = context[:-1]
        train.loc[uid, 'summary'] = data['label'][agenda]['summary']
        uid += 1

test = pd.DataFrame(columns=['uid', 'title', 'region', 'context'])
uid = 2000
for data in TEST_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        test.loc[uid, 'uid'] = uid
        test.loc[uid, 'title'] = data['title']
        test.loc[uid, 'region'] = data['region']
        test.loc[uid, 'context'] = context[:-1]
        uid += 1

sub = pd.read_csv('data/sample_submission.csv')
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [5]:
test

,uid,title,region,context
0,2000,제235회 본회의 제1차(2012.06.21.),음성,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 지금부터 음성군의회 제235회 ...
1,2001,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제1항, 음성군의회 제235회 제1차 정례회 회기결정의 건을 상정합니다. ..."
2,2002,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제2항, 회의록 서명의원 선출의 건을 상정합니다. 제235회 제1차 정례회..."
3,2003,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제3항, 예산결산특별위원회 구성의 건을 상정합니다. 예산결산특별위원회 구성..."
4,2004,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제4항, 환경분야 현지확인 특별위원회 구성결의안을 상정합니다. 대표발의하신..."
...,...,...,...,...
501,2501,제174회 본회의 제2차(2006.10.16.),음성,"의사일정 제1항, 제174회 음성군의회 임시회 회기 연장의 건을 상정합니다. 제17..."
502,2502,제174회 본회의 제2차(2006.10.16.),음성,"의사일정 제2항, 농협 축산물공판장 이전관련 현지확인 특별위원회 활동기간 연장의 건..."
503,2503,제174회 본회의 제2차(2006.10.16.),음성,"의사일정 제3항, 음성군 대소도서관 관리운영 조례안을 상정합니다. 문화공보과장께서는..."
504,2504,제174회 본회의 제2차(2006.10.16.),음성,"의사일정 제4항, 제4기 음성군 지역보건 의료계획안에 대해서 보건소장께서는 나오셔서..."


In [8]:
sub['summary'] = test['context'].str[:2000].progress_apply(summarization)

  0%|          | 0/506 [00:00<?, ?it/s]c:\Users\KDH\venv\docsum\lib\site-packages\torch\_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ..\aten\src\ATen\native\BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
100%|██████████| 506/506 [29:59<00:00,  3.56s/it]


In [9]:
sub

,uid,summary
0,id_2000-AGENDA_1,성원이 되었으므로 지금부터 음성군의회 제235회 제1차 정례회 제1차 본회의를 개의...
1,id_2000-AGENDA_2,음성군의회 제235회 제1차 정례회 제1차 정례회 회기는 「음성군의회 회기와 그 운...
2,id_2000-AGENDA_3,제235회 제1차 정례회 회의록 서명의원으로는 의원님들이 사전에 양해해 주신대로 조...
3,id_2000-AGENDA_4,"예산결산특별위원회 운영기간은 6월 25일 하루 동안 운영하고자 하며, 2011년도 ..."
4,id_2000-AGENDA_5,제235회 제1차 정례회 제1차 본회의를 맞이하여 음성군 환경분야 현지확인을 발의하...
...,...,...
501,id_2096-AGENDA_2,제174회 음성군의회 임시회 회기는 당초 지난 10월 10일부터 오늘까지 7일간으로...
502,id_2096-AGENDA_3,"정태완 위원장님, 본 의원은 지난 10월 10일 제1차 본회의시 농협 축산물공판장..."
503,id_2096-AGENDA_4,"대소도서관의 신축에 따라 관리운영의 근거를 마련하고자 하는 사항으로 관장의 지정, ..."
504,id_2096-AGENDA_5,제4기 음성군 지역보건 의료계획은 지역보건의료계획은 지역보건법 제3조 및 동법시행령...


In [10]:
sub.to_csv('kobart_submission.csv', index=False)